In [1]:
### no brand names and search terms vector

In [1]:
%load_ext Cython

In [2]:
%%time
%%cython
from gensim.models import doc2vec
import nltk
import re
def split_sentence (sentence):
    return re.split('\W+',sentence)
    
class MyDocs(object):
    def __iter__(self):
        for i, text in enumerate(open("txt_ds.txt")): # doesn't use brand name
            yield doc2vec.LabeledSentence(words=split_sentence(text), tags=['%s' % i])
            # Train the doc2vec model
cdef desp = MyDocs()
model = doc2vec.Doc2Vec(desp, size = 200, window = 8, min_count = 5, workers = 4)
model.save('distance_pd_nst.model')

CPU times: user 11min 3s, sys: 23.4 s, total: 11min 27s
Wall time: 4min 37s


In [3]:
model = doc2vec.Doc2Vec.load('distance_pd_nst.model')

In [4]:
import numpy as np
st_arrays = np.zeros((240760, 200))
for i,text in enumerate(open("txt_st.txt")): # input search terms 
    st_arrays[i]=model.infer_vector(split_sentence(text))   
st_arrays

array([[ 0.00544871,  0.01626157,  0.09603643, ..., -0.0141736 ,
         0.01279489,  0.03367273],
       [ 0.01602352,  0.095274  ,  0.04291017, ...,  0.01801155,
         0.00217703,  0.04987503],
       [-0.02138776,  0.01202906,  0.02334139, ..., -0.03393132,
         0.06721734,  0.04110518],
       ..., 
       [-0.0082629 , -0.00570589,  0.02126374, ...,  0.03695993,
         0.05154884,  0.00396368],
       [-0.02085695,  0.03738638,  0.02295393, ..., -0.0682317 ,
        -0.03370527,  0.00965658],
       [-0.00194651,  0.0530374 , -0.00177822, ..., -0.01150071,
        -0.00709637,  0.010651  ]])

In [5]:
## calculate the cosin distance between 2 vectors to generate the 3rd features
# Compute the cosine similarity values between the input text and all archived reviews
# cossims_with_input = map(lambda v: cossim(input_vec, v), model.docvecs) 
# need to chaneg the code into rows: calculate the cosin distance between the vectors in same rows
import numpy as np
import pandas as pd
# Calculate the cosine similarity between two vecotrs 
def cossim(v1, v2):
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2))

cossims_with_st=np.zeros((240760,1))
for i in range(240760):
    cossims_with_st[i] = cossim(st_arrays[i], model.docvecs[i])

In [6]:
np.savetxt("distance_pd_nst.csv",cossims_with_st, delimiter=",")

In [7]:
coswithst=pd.read_csv("distance_pd_nst.csv",header=None)
coswithst


,0
0,0.066799
1,-0.092491
2,0.051947
3,-0.023768
4,0.032217
5,0.166925
6,0.139467
7,0.092244
8,0.208593
9,0.102395


In [ ]:
cws_trn=coswithst.iloc[:num_train]
cws_tst=coswithst.iloc[num_train:]